In [1]:
# Test Wiener Filter's performance for dehazing images alone

import cv2
import os
import numpy as np
import scipy
from scipy.signal import convolve

In [2]:
def psnr(im1, im2):
    mse = np.mean((im1-im2)**2)
    if mse == 0:
        return 100
    return 20 * np.log10(255 / np.sqrt(mse))

def ssim(im1, im2):
    from skimage.measure import compare_ssim
    if im1.dtype!=im2.dtype:
        im1 = np.int64(im1)
        im2 = np.int64(im2)
    if im1.shape>1:
        return compare_ssim(im1, im2, multichannel=True)
    else:
        return compare_ssim(im1,im2)

In [3]:
import numpy as np
import cv2
from scipy.signal import wiener

def estimate_atmospheric_light(img, dark_channel):
    """Estimate atmospheric light using the brightest pixels in the dark channel."""
    h, w = dark_channel.shape
    num_pixels = max(1, (h * w) // 1000)  # Top 0.1% brightest pixels
    flat_dark = dark_channel.flatten()
    flat_img = img.reshape(-1, 3)
    indices = np.argsort(flat_dark)[-num_pixels:]
    return np.mean(flat_img[indices], axis=0)

def dark_channel(img, patch_size=15):
    """Compute the dark channel of the image."""
    min_channel = np.min(img, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch_size, patch_size))
    return cv2.erode(min_channel, kernel)

def estimate_transmission(img, A, omega=0.95):
    """Estimate the transmission map."""
    norm_img = img / A
    return 1 - omega * dark_channel(norm_img)

def apply_wiener_filter(transmission, kernel_size=(5,5)):
    """Refine the transmission map using the Wiener filter."""
    return wiener(transmission, kernel_size)

def recover_image(img, t, A, t_min=0.1):
    """Recover the dehazed image using the refined transmission map."""
    t = np.maximum(t, t_min)
    J = (img - A) / t[..., np.newaxis] + A
    return np.clip(J * 255, 0, 255).astype(np.uint8)

def dehaze_image(image_path):
    """Perform image dehazing using the Wiener filter."""
    img = cv2.imread(image_path).astype(np.float32) / 255.0
    dark = dark_channel(img)
    A = estimate_atmospheric_light(img, dark)
    t = estimate_transmission(img, A)
    t_refined = apply_wiener_filter(t)
    dehazed = recover_image(img, t_refined, A)
    return dehazed

# Load and dehaze an image
image_path = "hazy_image.jpg"
dehazed_image = dehaze_image(image_path)

# Save or display the dehazed image
cv2.imwrite("dehazed_image.jpg", dehazed_image)


AttributeError: 'NoneType' object has no attribute 'astype'